Задача 14 (Полёт ракеты)

Проверка

In [1]:
import numpy as np
import plotly.graph_objects as go


def test_rhs(t, y):
    return np.array([np.exp(2*t)*y[1], -np.exp(-2*t)*y[0]])


def test_exact(t):
    return np.array([np.exp(t), np.exp(-t)])


def rk3_step(f, t, y, h):
    k1 = f(t, y)
    k2 = f(t + 0.5*h, y + 0.5*h*k1)
    k3 = f(t + h,   y - h*k1 + 2*h*k2)
    return y + (h/6)*(k1 + 4*k2 + k3)


hs = np.linspace(0.1, 0.01, 10)
t_end = 4.0

errors = []
Cs = []
alphas = []


for h in hs:
    t = 0.0
    y = np.array([1.0, 1.0])
    while t < t_end:
        h_step = h if t + h <= t_end else t_end - t
        y = rk3_step(test_rhs, t, y, h_step)
        t += h_step
    err = np.max(np.abs(y - test_exact(t_end)))
    errors.append(err)
    Cs.append(err / h**3)

for i in range(len(hs)-1):
    α = np.log(errors[i]/errors[i+1]) / np.log(hs[i]/hs[i+1])
    alphas.append(α)

for i in Cs:
    print(np.log(i))
print(f"{'h':>8} {'e(h)':>12} {'e/h^3':>12} {'α':>8}")
for i, h in enumerate(hs):
    a_str = f"{alphas[i]:.4f}" if i < len(alphas) else "   --  "
    print(f"{h:8.10f} {errors[i]:12.10f} {Cs[i]:12.10f} {a_str}")

fig1 = go.Figure(go.Scatter(
    x=hs, y=errors, mode='lines+markers', name='e(h)'
))
fig1.update_layout(
    title='max|e| vs h (RK3 тест на [0,4])',
    xaxis=dict(title='h', type='log', tickformat='.10f'),
    yaxis=dict(title='max|e|', type='log', tickformat='.10f')
)
fig1.show()

fig2 = go.Figure(go.Scatter(
    x=hs, y=Cs, mode='lines+markers', name='e/h^3'
))
fig2.update_layout(
    title='e(h)/h^3 vs h (проверка порядка)',
    xaxis=dict(title='h', type='log', tickformat='.10f'),
    yaxis=dict(title='e/h^3', type='log', tickformat='.10f')
)
fig2.show()

3.2272718436925985
3.225049606610152
3.243086396133263
3.2483250735077185
3.2514125184996345
3.2669230990176823
3.274893202616876
3.2803831524952547
3.290860571176928
3.29885438943152
       h         e(h)        e/h^3        α
0.1000000000 0.0252107841 25.2107841060 3.0211
0.0900000000 0.0183378652 25.1548219703 2.8469
0.0800000000 0.0131136771 25.6126506677 2.9608
0.0700000000 0.0088312824 25.7471791493 2.9800
0.0600000000 0.0055785877 25.8267949901 2.9149
0.0500000000 0.0032788133 26.2305063787 2.9643
0.0400000000 0.0016921857 26.4404015641 2.9809
0.0300000000 0.0007178208 26.5859572245 2.9742
0.0200000000 0.0002149278 26.8659737932 2.9885
0.0100000000 0.0000270816 27.0815961798    --  


Задача

In [1]:
import numpy as np
import plotly.graph_objects as go


def mass_and_thrust(t, m0, mT, ts, T_const):
    q = mT/ts if t <= ts else 0.0
    rem_fuel = max(0.0, mT - q*t)
    m = (m0 - mT) + rem_fuel
    return m, q, T_const


def rocket_rhs(t, state, params):
    x, y, v1, v2 = state
    g, C, S, rho, m0, mT, ts, T_const = params

    m, q, T = mass_and_thrust(t, m0, mT, ts, T_const)

    # абсолютная скорость
    v = np.sqrt(v1**2 + v2**2)

    # ускорение от тяги и сопротивления
    drag_acc = (T - 0.5*C*rho*S*v**2) / m

    dx_dt = v1
    dy_dt = v2
    dv1_dt = drag_acc * (v1/v if v != 0 else 0.0) - (q/m)*v1
    dv2_dt = drag_acc * (v2/v if v != 0 else 0.0) - (q/m)*v2 - g

    return np.array([dx_dt, dy_dt, dv1_dt, dv2_dt])


def rk3_step(f, t, y, h, params):
    k1 = f(t,       y, params)
    k2 = f(t+0.5*h, y+0.5*h*k1, params)
    k3 = f(t + h,  y - h*k1 + 2*h*k2, params)
    return y + (h/6.0)*(k1 + 4*k2 + k3)


flag = bool(int(input("Используем параметры из задания? (1-Да,0-Нет) ")))
if flag:
    g, C, S, rho = 9.81, 0.2, 0.25, 1.29
    m0, mT, ts = 30., 15., 4.
    T_const, v0 = 5., 50.
    theta_start, theta_end = 30, 80
    theta_step = int(input("Шаг проверки угла, °: "))
    h = float(input("Шаг интегрирования h, с: "))
else:
    m0 = float(input("m0 [кг]: "))
    mT = float(input("mT [кг]: "))
    ts = float(input("ts [с]: "))
    C = float(input("C: "))
    rho = float(input("rho [кг/м3]: "))
    S = float(input("S [м2]: "))
    g = float(input("g [м/с2]: "))
    T_const = float(input("T [Н]: "))
    v0 = float(input("v0 [м/с]: "))
    h = float(input("h [с]: "))
    theta_start = int(input("θ_start [°]: "))
    theta_end = int(input("θ_end   [°]: "))
    theta_step = int(input("Шаг угла, °: "))

params = (g, C, S, rho, m0, mT, ts, T_const)

traj_fig = go.Figure()
L_vals, theta_vals = [], []

for deg in range(theta_start, theta_end+1, theta_step):
    theta = np.radians(deg)
    state = np.array([0.0, 0.0, v0*np.cos(theta), v0*np.sin(theta)])
    t = 0.0
    traj = [[0.0, 0.0]]
    while state[1] >= 0.0:
        state = rk3_step(rocket_rhs, t, state, h, params)
        t += h
        traj.append([state[0], state[1]])

    traj = np.array(traj)
    traj_fig.add_trace(go.Scatter(
        x=traj[:, 0], y=traj[:, 1], mode='lines', name=f"{deg}°"
    ))
    L_vals.append(traj[-1, 0])
    theta_vals.append(deg)

best = np.argmax(L_vals)
print(
    f"\nОптимальный угол: {theta_vals[best]}°, дальность: {L_vals[best]:.2f} м\n")

traj_fig.update_layout(
    title="Траектории для разных углов",
    xaxis_title="x, м", yaxis_title="y, м"
)
traj_fig.show()

L_fig = go.Figure(go.Scatter(
    x=theta_vals, y=L_vals, mode='lines+markers'
))
L_fig.update_layout(
    title="Дальность полёта L(θ)",
    xaxis_title="θ, °", yaxis_title="L, м"
)
L_fig.show()


Оптимальный угол: 35°, дальность: 133.35 м

